<a href="https://colab.research.google.com/github/adammoss/MLiS2/blob/master/examples/rnn/cbow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

TensorFlow 2.x selected.


In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import numpy as np
import gensim

In [0]:
tf.random.set_seed(1)

In [5]:
print(tf.__version__)

2.1.0


In [0]:
path = keras.utils.get_file('alice.txt', origin='http://www.gutenberg.org/files/11/11-0.txt')
corpus = open(path).readlines()[:500]
corpus = [sentence for sentence in corpus if sentence.count(' ') >= 2]

tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
corpus = tokenizer.texts_to_sequences(corpus)
nb_samples = sum(len(s) for s in corpus)
word_dim = len(tokenizer.word_index) + 1
embedding_dim = 100
window_size = 2

In [20]:
print("Found %d unique words" % word_dim)

Found 1103 unique words


In [0]:
def generate_data(corpus, window_size, word_dim):
  maxlen = window_size*2
  for words in corpus:
    L = len(words)
    for index, word in enumerate(words):
      contexts = []
      labels   = []            
      s = index - window_size
      e = index + window_size + 1
      contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
      labels.append(word)
      x = keras.preprocessing.sequence.pad_sequences(contexts, maxlen=maxlen)
      y = keras.utils.to_categorical(labels, word_dim)
      yield (x, y)

In [0]:
cbow = keras.Sequential()
cbow.add(layers.Embedding(input_dim=word_dim, output_dim=embedding_dim, input_length=window_size*2))
cbow.add(layers.Lambda(lambda x: keras.backend.mean(x, axis=1), output_shape=(embedding_dim,)))
cbow.add(layers.Dense(word_dim, activation='softmax'))

In [33]:
cbow.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 4, 100)            110300    
_________________________________________________________________
lambda_3 (Lambda)            (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1103)              111403    
Total params: 221,703
Trainable params: 221,703
Non-trainable params: 0
_________________________________________________________________


In [0]:
cbow.compile(loss='categorical_crossentropy', optimizer='adam')

In [0]:
EPOCHS = 10

In [35]:
for epoch in range(EPOCHS):
    loss = 0.
    for x, y in generate_data(corpus, window_size, V):
        loss += cbow.train_on_batch(x, y)
    print(epoch, loss)

0 30694.245127916336
1 26644.468227624893
2 24520.527456611395
3 22430.948384150863
4 20346.574242837727
5 18315.21217214316
6 16384.445414680988
7 14579.34523926489
8 12909.14413444139
9 11377.445527866483


In [44]:
f = open('vectors.txt' ,'w')
f.write('{} {}\n'.format(word_dim-1, embedding_dim))

9

In [0]:
vectors = cbow.get_weights()[0]
for word, i in tokenizer.word_index.items():
    str_vec = ' '.join(map(str, list(vectors[i, :])))
    f.write('{} {}\n'.format(word, str_vec))
f.close()

In [46]:
w2v = gensim.models.KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [47]:
w2v.most_similar(positive=['alice'])

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('severely', 0.4334946274757385),
 ('anxiously', 0.37974250316619873),
 ('cry', 0.3641851544380188),
 ('hurt', 0.3614460527896881),
 ('indeed', 0.3588726818561554),
 ('“well', 0.3446401357650757),
 ('saying', 0.3297501504421234),
 ('hadn’t', 0.3283589780330658),
 ('child', 0.32811883091926575),
 ('she', 0.32219991087913513)]